In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from keras_tuner.tuners import RandomSearch

from keras.layers import Input
from keras.models import Model

2024-04-28 21:52:47.660398: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-28 21:52:48.154853: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
BATCH_SIZE = 512

In [3]:
AS_dataset = pd.read_csv('./hairpin_filter_dataset.csv', encoding='utf-8')
full_X = AS_dataset.loc[::2,'freq':'S1'].to_numpy(dtype = np.float32)
full_y = AS_dataset.loc[::2,'S21r':'S21i'].to_numpy(dtype = np.float32)

In [4]:
# 归一化
# full_X[0] = (full_X[0] - 0.1) / (9.0 - 0.1)
full_X[:, 1] = (full_X[:, 1] - 200) / (2000 - 200)
full_X[:, 2] = (full_X[:, 2] - 200) / (2000 - 200)
full_X[:, 3] = (full_X[:, 3] - 200) / (2000 - 200)
full_X[:, 4] = (full_X[:, 4] - 200) / (4500 - 200)
full_X[:, 5] = (full_X[:, 5] - 200) / (4500 - 200)
full_X[:, 6] = (full_X[:, 6] - 200) / (4500 - 200)
full_X[:, 7] = (full_X[:, 7] - 200) / (4500 - 200)
full_X[:, 8] = (full_X[:, 8] - 9000) / (12000 - 9000)
full_X[:, 9] = (full_X[:, 9] - 100) / (300 - 100)

In [5]:
X_train, X_vali, y_train, y_vali = train_test_split(full_X, full_y, test_size=0.05, random_state=0)

In [6]:
dataset_train = tf.data.Dataset.from_tensor_slices((X_train, y_train))
dataset_train = dataset_train.shuffle(buffer_size=X_train.shape[0])
dataset_train = dataset_train.batch(BATCH_SIZE)
dataset_train = dataset_train.prefetch(tf.data.experimental.AUTOTUNE)

2024-04-28 21:52:50.507213: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9604 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:17:00.0, compute capability: 7.5
2024-04-28 21:52:50.507700: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 9621 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:65:00.0, compute capability: 7.5


In [7]:
dataset_vali = tf.data.Dataset.from_tensor_slices((X_vali, y_vali))
dataset_vali = dataset_vali.shuffle(buffer_size=X_vali.shape[0])
dataset_vali = dataset_vali.batch(BATCH_SIZE)
dataset_vali = dataset_vali.prefetch(tf.data.experimental.AUTOTUNE)

In [8]:
class MLP(tf.keras.Model):
    def __init__(self, sematicSize, numLayers, numUnits, useReLU, useNorm):
        super().__init__()
        self.emb = tf.keras.layers.Embedding(101, 9)
        
        self.dense1 = tf.keras.layers.Dense(units=sematicSize)
        self.dense2 = tf.keras.layers.Dense(units=sematicSize)

        self.hiddenLayers = []
        for i in range(numLayers):
            self.hiddenLayers.append(tf.keras.layers.Dense(units=numUnits[i], activation=useReLU[i]))
            if useNorm[i]:
                self.hiddenLayers.append(tf.keras.layers.BatchNormalization())

        self.outputLayer = tf.keras.layers.Dense(units=2)

    def call(self, inputs):
        f = tf.cast(tf.round(inputs[:, 0] * 10), tf.int8)
        g = inputs[:, 1:]
        
        x1 = self.dense1(self.emb(f))
        x2 = self.dense2(g)
        x = x1 + x2
        
        for i in range(len(self.hiddenLayers)):
            x = self.hiddenLayers[i](x)

        output = self.outputLayer(x)
        return output

In [9]:
# 定义模型构建函数
def build_model(hp):
    
    sematicSize = hp.Int('units_first', min_value=32, max_value=1024, step=32)
    numLayers = hp.Int('num_layers', 1, 5)
    numUnits = [hp.Int('units_' + str(i), min_value=32, max_value=1024, step=32) for i in range(numLayers)]
    useReLU = ['leaky_relu' if hp.Boolean('use_relu' + str(i)) else None for i in range(numLayers)]
    useNorm = [hp.Boolean('use_batchnorm' + str(i)) for i in range(numLayers)]
    
    model = MLP(sematicSize=sematicSize, numLayers=numLayers, numUnits=numUnits, useReLU=useReLU, useNorm=useNorm)

    model.compile(
        optimizer=tf.keras.optimizers.Adam(hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
        loss='mse')
    
    return model

In [10]:
# 创建 tuner
tuner = RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=1024, # 根据实际情况调整试验次数
    directory='my_dir',
    project_name='mlp_project1')

In [ ]:
# 开始搜索过程
tuner.search(dataset_train,
             epochs=50,
            validation_data=dataset_vali)

Trial 160 Complete [00h 03m 22s]
val_loss: 0.05985086411237717

Best val_loss So Far: 0.0024111841339617968
Total elapsed time: 10h 35m 28s

Search: Running Trial #161

Value             |Best Value So Far |Hyperparameter
480               |256               |units_first
4                 |5                 |num_layers
832               |640               |units_0
True              |True              |use_relu0
True              |False             |use_batchnorm0
0.01              |0.001             |learning_rate
32                |736               |units_1
768               |672               |units_2
False             |True              |use_relu1
True              |True              |use_relu2
True              |True              |use_batchnorm1
True              |False             |use_batchnorm2
96                |576               |units_3
992               |32                |units_4
False             |False             |use_relu3
False             |True              |use_relu